<a href="https://colab.research.google.com/github/luqmansen/notebook-collection/blob/master/kNN_cars_classifier_standford_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Mata Kuliah Data Mining**
## Klasifikasi Data Citra menggunakan KNN

1. Luqman Setyo / Nugroho 21120117120008

2. Heidy novendra / 21120117130035

3. Faiz Noerdiyan / 21120117130060

4. Chandra Purnama / 21120117130070

Sumber dataset

Stanford Car Dataset by classes folder

https://www.kaggle.com/jutrera/stanford-car-dataset-by-classes-folder

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/knn-Cars-classifier'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/knn-Cars-classifier


In [2]:
!pwd

/content/drive/My Drive/Colab Notebooks/knn-Cars-classifier


In [0]:
!wget 'https://storage.googleapis.com/kaggle-data-sets/31559/46697/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586698258&Signature=EYYFx3xlTVzpnBRMqIndbj%2FsoOE4viyYWJABJILIwA99fIN7skQWjOcS%2FOGNrb8i62jGPXdnJ%2F667yom0LNJlWO01msTHM1kiNRMs9ZxupFbwgwDCb4Dena2D%2FSMSmLfut5Sq4Yc0uJ6YU2znfQLOgJP5x8lFZrqPQL9bGRnESynAp0hZ%2BSCcqIIJ4flBcNFj%2F7bsBUZWLA%2BHQx%2FteI8NMuN%2BnFYXI%2FGWdQvDvoCLzqTTwZ3zHdDuHbzFM4MgZ7XWk8Xsgj5wBiGIRyq5ndLDY4UfnnERyOnXgmrL5empYwMNQIwNIT%2B6q6jHtAlYgV1w1bxQ29sMuZ4q%2FKpqnJdFw%3D%3D&response-content-disposition=attachment%3B+filename%3Dstanford-car-dataset-by-classes-folder.zip'

The name is too long, 556 chars total.
Trying to shorten...
New name is archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586698258&Signature=EYYFx3xlTVzpnBRMqIndbj%2FsoOE4viyYWJABJILIwA99fIN7skQWjOcS%2FOGNrb8i62jGPXdnJ%2F667yom0LNJlWO01msTHM1kiNRMs9ZxupFbwgwDCb4Dena2D%2FSMS.
--2020-04-09 13:36:05--  https://storage.googleapis.com/kaggle-data-sets/31559/46697/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586698258&Signature=EYYFx3xlTVzpnBRMqIndbj%2FsoOE4viyYWJABJILIwA99fIN7skQWjOcS%2FOGNrb8i62jGPXdnJ%2F667yom0LNJlWO01msTHM1kiNRMs9ZxupFbwgwDCb4Dena2D%2FSMSmLfut5Sq4Yc0uJ6YU2znfQLOgJP5x8lFZrqPQL9bGRnESynAp0hZ%2BSCcqIIJ4flBcNFj%2F7bsBUZWLA%2BHQx%2FteI8NMuN%2BnFYXI%2FGWdQvDvoCLzqTTwZ3zHdDuHbzFM4MgZ7XWk8Xsgj5wBiGIRyq5ndLDY4UfnnERyOnXgmrL5empYwMNQIwNIT%2B6q6jHtAlYgV1w1bxQ29sMuZ4q%2FKpqnJdFw%3D%3D&response-content-disposition=attachment%3B+filename%3Dstanford-car-dataset-by-classes-folder.zip
Resolving storage.googlea

In [0]:
!unzip -q archive*

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
import time

In [0]:
def image_to_feature_vector(image, size=(32, 32)):
	# Konversi gambar menjadi berukuran 32x32 
    
  # return berupa list of number 32x32x3 (3 = channel rgb) = list of 3.072 
	return cv2.resize(image, size).flatten()

In [0]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# konversi gambar ke hsv
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  # menghitung histogram gambar
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
	
  # Normalisasi histogram
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	else:
		cv2.normalize(hist, hist)
  
	# return berupa feature vector 512-d (8 untuk tiap H,S,V sehingga 8x8x8)
	return hist.flatten()

In [0]:
imagePaths = list(paths.list_images("car_data/car_data/train/"))
# initialize the raw pixel intensities matrix, the features matrix, and labels list
rawImages = []
features = []
labels = []

In [0]:
test_imagePaths = list(paths.list_images("car_data/car_data/test/"))
# initialize the raw pixel intensities matrix, the features matrix, and labels list
test_rawImages = []
test_features = []
test_labels = []

In [0]:
def extractor(raw_list, feature_list, label_list, path):
  print("[INFO] proccessing {}".format("-".join(path[0].split("/")[1:3])))
  # loop over the input images
  for (i, imagePath) in enumerate(path):
    # load the image and extract the class label 
    image = cv2.imread(imagePath)
    label = imagePath.split("/")[-2]
    # extract raw pixel intensity "features", followed by a color
    # histogram to characterize the color distribution of the pixels in the image
    pixels = image_to_feature_vector(image)
    hist = extract_color_histogram(image)
    # update the raw images, features, and labels matricies, respectively
    raw_list.append(pixels)
    feature_list.append(hist)
    label_list.append(label)
    # show an update every 1,000 images
    if i > 0 and i % 1000 == 0:
      print("[INFO] processed {}/{}".format(i, len(path)))

In [55]:
extractor(rawImages, features, labels,imagePaths)

[INFO] proccessing car_data-train
[INFO] processed 1000/8144
[INFO] processed 2000/8144
[INFO] processed 3000/8144
[INFO] processed 4000/8144
[INFO] processed 5000/8144
[INFO] processed 6000/8144
[INFO] processed 7000/8144
[INFO] processed 8000/8144


In [56]:
extractor(test_rawImages, test_features, test_labels, test_imagePaths)

[INFO] proccessing car_data-test
[INFO] processed 1000/8041
[INFO] processed 2000/8041
[INFO] processed 3000/8041
[INFO] processed 4000/8041
[INFO] processed 5000/8041
[INFO] processed 6000/8041
[INFO] processed 7000/8041
[INFO] processed 8000/8041


In [57]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
test_rawImages = np.array(test_rawImages)
test_features = np.array(test_features)
test_labels = np.array(test_labels)
print("[INFO] pixels matrix: {:.2f}MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(features.nbytes / (1024 * 1000.0)))
print("[INFO] test pixels matrix: {:.2f}MB".format(test_rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] test features matrix: {:.2f}MB".format(test_features.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 24.43MB
[INFO] features matrix: 16.29MB
[INFO] test pixels matrix: 24.12MB
[INFO] test features matrix: 16.08MB


In [58]:
# train and evaluate a k-NN classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1,	n_jobs=-1)
model.fit(rawImages, labels)
acc = model.score(test_rawImages, test_labels)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 2.70%


In [59]:
# train and evaluate a k-NN classifer on the histogram
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1,	n_jobs=-1)
model.fit(features, labels)
acc = model.score(test_features, test_labels)
print("[INFO] histogram pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] histogram pixel accuracy: 4.43%


## Hyperparameter tunning

n_neghbors : jumlah NN k (dari 0 hingga 29, exclusive)

metrics : distance function metrics

In [0]:
params = {"n_neighbors": np.arange(1, 63, 2),
	"metric": ["euclidean", "cityblock"]}

In [0]:
model = KNeighborsClassifier(n_jobs=-1)

In [74]:
# tune the hyperparameters via a randomized search
print("[INFO] tuning hyperparameters via randomized search")
grid = RandomizedSearchCV(model, params, verbose=1)
start = time.time()
grid.fit(features, labels)
# evaluate the best randomized searched model on the testing data
print("[INFO] randomized search took {:.2f} seconds".format(time.time() - start))
acc = grid.score(test_features, test_labels)
print("[INFO] randomized search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] randomized search best parameters: {}".format(grid.best_params_))

[INFO] tuning hyperparameters via randomized search
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  6.6min finished


[INFO] randomized search took 397.68 seconds
[INFO] randomized search accuracy: 3.12%
[INFO] randomized search best parameters: {'n_neighbors': 11, 'metric': 'cityblock'}


In [0]:
# Grid Search tuning
print("[INFO] tuning hyperparameters via grid search")
grid = GridSearchCV(model, params, verbose=1)
start = time.time()
grid.fit(features, labels)
# evaluate the best grid searched model on the testing data
print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
acc = grid.score(test_features, test_labels)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] tuning hyperparameters via grid search
Fitting 5 folds for each of 62 candidates, totalling 310 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
